## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import json

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [15]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vila,PT,42.03,-8.16,46.00,91,0,1.01,clear sky
1,1,Ribeira Grande,PT,38.52,-28.70,57.20,50,40,3.36,scattered clouds
2,2,Vaini,TO,-21.20,-175.20,82.40,83,40,19.46,scattered clouds
3,3,Worthington,US,40.09,-83.02,51.01,61,90,4.70,overcast clouds
4,4,Atuona,PF,-9.80,-139.03,78.78,71,32,19.08,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
#Ask the customer to add a minimum and maximum temperature value
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
ideal_temp_df=city_data_df.loc[(city_data_df["Max Temp"] >=min_temp)&\
                                (city_data_df["Max Temp"] <=max_temp)]
ideal_temp_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Vaini,TO,-21.20,-175.20,82.40,83,40,19.46,scattered clouds
4,4,Atuona,PF,-9.80,-139.03,78.78,71,32,19.08,scattered clouds
6,6,Alofi,NU,-19.06,-169.92,82.40,88,40,9.17,scattered clouds
9,9,Hilo,US,19.73,-155.09,75.99,78,75,6.93,light rain
12,12,Faanui,PF,-16.48,-151.75,80.67,74,90,3.18,overcast clouds
13,13,Farafenni,GM,13.57,-15.60,76.32,18,0,8.21,clear sky
14,14,Arraial Do Cabo,BR,-22.97,-42.03,77.00,83,40,18.34,scattered clouds
16,16,Kapaa,US,22.08,-159.32,80.60,69,40,19.46,scattered clouds
17,17,Sotouboua,TG,8.57,0.98,78.80,78,0,4.16,clear sky
20,20,Ambilobe,MG,-13.20,49.05,76.69,80,74,5.70,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
ideal_temp_df.count()

City_ID                206
City                   206
Country                206
Lat                    206
Lng                    206
Max Temp               206
Humidity               206
Cloudiness             206
Wind Speed             206
Current Description    206
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# ideal_temp_df=city_data_df.loc[(city_data_df["Max Temp"] >=min_temp)&\
#                                 (city_data_df["Max Temp"] <=max_temp)].dropna()
# ideal_temp_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = ideal_temp_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Vaini,TO,82.40,scattered clouds,-21.20,-175.20,
4,Atuona,PF,78.78,scattered clouds,-9.80,-139.03,
6,Alofi,NU,82.40,scattered clouds,-19.06,-169.92,
9,Hilo,US,75.99,light rain,19.73,-155.09,
12,Faanui,PF,80.67,overcast clouds,-16.48,-151.75,
13,Farafenni,GM,76.32,clear sky,13.57,-15.60,
14,Arraial Do Cabo,BR,77.00,scattered clouds,-22.97,-42.03,
16,Kapaa,US,80.60,scattered clouds,22.08,-159.32,
17,Sotouboua,TG,78.80,clear sky,8.57,0.98,
20,Ambilobe,MG,76.69,broken clouds,-13.20,49.05,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"]= f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels=requests.get(base_url, params=params).json()
    
    #6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"]= hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")


Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [9]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Vaini,TO,82.40,scattered clouds,-21.20,-175.20,Keleti Beach Resort
4,Atuona,PF,78.78,scattered clouds,-9.80,-139.03,Villa Enata
6,Alofi,NU,82.40,scattered clouds,-19.06,-169.92,Taloa Heights
9,Hilo,US,75.99,light rain,19.73,-155.09,Hilo Hawaiian Hotel
12,Faanui,PF,80.67,overcast clouds,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa


In [10]:
# 7. Drop the rows where there is no Hotel Name.
new_hotel_df = hotel_df[hotel_df["Hotel Name"]!= ""]
new_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Vaini,TO,82.40,scattered clouds,-21.20,-175.20,Keleti Beach Resort
4,Atuona,PF,78.78,scattered clouds,-9.80,-139.03,Villa Enata
6,Alofi,NU,82.40,scattered clouds,-19.06,-169.92,Taloa Heights
9,Hilo,US,75.99,light rain,19.73,-155.09,Hilo Hawaiian Hotel
12,Faanui,PF,80.67,overcast clouds,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
...,...,...,...,...,...,...,...
650,Vilhena,BR,78.80,scattered clouds,-12.74,-60.15,Hotel Mirage
657,San Marcos De Colon,HN,93.20,scattered clouds,13.43,-86.80,Hotel Colonial
662,Manta,EC,77.00,few clouds,-0.95,-80.73,Balandra Hotel
663,Sarangani,PH,83.84,very heavy rain,5.40,125.46,Ballistic Islet


In [11]:
# 8a. Create the output File (CSV)
#export to csv
export_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
new_hotel_df.to_csv(export_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description </dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in new_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = new_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig= gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))